In [1]:
import pandas as pd 
import numpy as np 
import plotly.io as pio

In [2]:
lof = pd.read_csv('../Datathon2020data/lof.csv')
kms = pd.read_csv('../Datathon2020data/kmeans.csv')
iso = pd.read_csv('../Datathon2020data/iso.csv')
# lof = pd.read_csv('~/Desktop/lof.csv')
# kms = pd.read_csv('~/Desktop/kmeans.csv')
# iso = pd.read_csv('~/Desktop/iso.csv')

In [3]:
lof['lof'].value_counts(normalize=True)

0    0.864979
1    0.135021
Name: lof, dtype: float64

In [4]:
kms['kmeans'].value_counts(normalize=True)

0    0.914923
1    0.085077
Name: kmeans, dtype: float64

In [5]:
iso['iso'].value_counts(normalize=True)

0    0.899998
1    0.100002
Name: iso, dtype: float64

### Merging all results

In [6]:
all_df = pd.concat([lof, kms[['kmeans']], iso[['iso']]], axis=1)

In [7]:
all_df['mutual_real'] = np.where(((all_df['lof']==1)&(all_df['kmeans']==1)&(all_df['iso']==1)), '1','0')
all_df['union_real'] = np.where((all_df['lof']==1)|(all_df['kmeans']==1)|(all_df['iso']==1), '1','0')

In [8]:
all_df['mutual_real'].value_counts(normalize=True)

0    0.994304
1    0.005696
Name: mutual_real, dtype: float64

In [9]:
all_df['union_real'].value_counts(normalize=True)

0    0.743303
1    0.256697
Name: union_real, dtype: float64

In [10]:
all_df.drop(['kmeans', 'iso'], axis=1, inplace=True)

### Merge back with non_fraud

In [14]:
claims = pd.read_csv('../cleaned_data/claims.csv')
#claims = pd.read_csv("/Applications/datathon/Datathon2020data/claims.csv")

#small fix from the last notebook 
claims['Age'] = claims['Age']//365
claims['Is_Dead'] = np.where(claims['Death_age'].isna(), False, True)
claims['DeductibleAmt'] = np.where(claims['DeductibleAmt'].isna(), 0, claims['DeductibleAmt'])

claims['InpatientAnnualDeductibleAmt'] = np.where(claims['InpatientAnnualDeductibleAmt']==0,0,1)

#Single grouping - amount less than 500
claims['OutpatientAnnualDeductibleAmt'] = np.where(claims['OutpatientAnnualDeductibleAmt']<=500,0,1)

claims['DeductibleAmt'] = np.where(claims['DeductibleAmt']==0, 0, 1)

#small fix
claims['Age'] = claims['Age']//365
claims['Is_Dead'] = np.where(claims['Death_age'].isna(), False, True)
claims['DeductibleAmt'] = np.where(claims['DeductibleAmt'].isna(), 0, claims['DeductibleAmt'])

claims['InpatientAnnualDeductibleAmt'] = np.where(claims['InpatientAnnualDeductibleAmt']==0,0,1)

#Single grouping - amount less than 500
claims['OutpatientAnnualDeductibleAmt'] = np.where(claims['OutpatientAnnualDeductibleAmt']<=500,0,1)

claims['DeductibleAmt'] = np.where(claims['DeductibleAmt']==0, 0, 1)

In [15]:
### TO-DO: split reimbursement amount into groups

In [16]:
non_fraud = claims[claims['Fraud']==0].reset_index(drop=True)

In [17]:
non_fraud.drop(['Death_age','Fraud'],axis=1, inplace=True)

In [18]:
non_fraud['union_real'] = '-1'
non_fraud['mutual_real'] = '-1'
non_fraud['lof'] = -1

In [58]:
df = pd.concat([non_fraud, all_df], axis=0).reset_index(drop=True)

In [20]:
#df.to_csv('../web_app/data/final.csv')

### Visualization

In [22]:
import plotly.express as px

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 25 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   AmtReimbursed                     558211 non-null  int64 
 1   DeductibleAmt                     558211 non-null  int64 
 2   Is_inpatient                      558211 non-null  int64 
 3   Gender                            558211 non-null  int64 
 4   Race                              558211 non-null  int64 
 5   RenalDisease                      558211 non-null  int64 
 6   State                             558211 non-null  int64 
 7   County                            558211 non-null  int64 
 8   InpatientAnnualReimbursementAmt   558211 non-null  int64 
 9   InpatientAnnualDeductibleAmt      558211 non-null  int64 
 10  OutpatientAnnualReimbursementAmt  558211 non-null  int64 
 11  OutpatientAnnualDeductibleAmt     558211 non-null  int64 
 12  Nu

In [25]:
pd.options.plotting.backend = "plotly"

In [60]:
df['lof'].value_counts()

-1    345415
 0    184064
 1     28732
Name: lof, dtype: int64

In [61]:
df['lof'].replace({0:'0',-1:'-1',1:'1'},inplace=True)

In [62]:
real = df[df['lof']=='1']
potential = df[df['lof']=='0']
non = df[df['lof']=='-1']

In [63]:
real[['AmtReimbursed','ChronicDisease_Num',
    'Is_inpatient']].plot.scatter(
    x='ChronicDisease_Num', y='AmtReimbursed',
    size='ChronicDisease_Num', color='Is_inpatient')

In [64]:
df[df['AmtReimbursed']>8000].describe()

,AmtReimbursed,DeductibleAmt,Is_inpatient,Gender,Race,RenalDisease,State,County,InpatientAnnualReimbursementAmt,InpatientAnnualDeductibleAmt,OutpatientAnnualReimbursementAmt,OutpatientAnnualDeductibleAmt,NumOfClaimDays,Age,DiagnosisCode_Num,ProcedureCode_Num,ChronicDisease_Num,PhysiciansNum
count,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000
mean,17825.334962,0.965431,0.989987,1.577006,1.248242,0.287341,25.634402,378.164263,25537.051496,0.992729,2346.477530,0.181845,7.212361,42.957265,8.267314,1.042019,5.501848,1.912862
std,12304.620865,0.182691,0.099566,0.494049,0.702140,0.452535,15.216511,266.078823,19436.590384,0.084965,5336.190149,0.385728,6.610144,37.337062,1.799580,0.788371,2.149900,0.560390
min,8010.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10000.000000,1.000000,1.000000,1.000000,1.000000,0.000000,11.000000,150.000000,12030.000000,1.000000,180.000000,0.000000,3.000000,0.000000,8.000000,1.000000,4.000000,2.000000
50%,13000.000000,1.000000,1.000000,2.000000,1.000000,0.000000,25.000000,341.000000,19000.000000,1.000000,750.000000,0.000000,5.000000,63.000000,9.000000,1.000000,5.000000,2.000000
75%,20000.000000,1.000000,1.000000,2.000000,1.000000,1.000000,38.000000,570.000000,32000.000000,1.000000,2200.000000,0.000000,9.000000,76.000000,9.000000,1.000000,7.000000,2.000000
max,125000.000000,1.000000,1.000000,2.000000,5.000000,1.000000,54.000000,999.000000,161470.000000,1.000000,102960.000000,1.000000,36.000000,100.000000,10.000000,5.000000,11.000000,3.000000


In [65]:
amount= [(df['AmtReimbursed']<=300), 
         ((df['AmtReimbursed']>300)&(df['AmtReimbursed']<=1000)),
        ((df['AmtReimbursed']>1000)&(df['AmtReimbursed']<=8000)),
        ((df['AmtReimbursed']>8000)&(df['AmtReimbursed']<=20000)),
        (df['AmtReimbursed']>20000)]

In [66]:
level = ['under 300', '300-1k', '1k-8k', '8k-20k', '20k+']

In [67]:
df['Reimbursed_Level'] = np.select(amount, level)

In [68]:
df['Reimbursed_Level'].value_counts()

under 300    420392
300-1k        62943
1k-8k         58098
8k-20k        12810
20k+           3968
Name: Reimbursed_Level, dtype: int64

In [69]:
temp = df.groupby(['Reimbursed_Level', 'lof']).count().reset_index()

In [70]:
reim = temp.pivot(index='Reimbursed_Level', columns='lof',values='AmtReimbursed').reset_index()

In [71]:
reim.fillna(0, inplace=True)

In [72]:
reim['Reimbursed_Level'] = pd.Categorical(reim['Reimbursed_Level'],
                                         level)

In [73]:
reim.sort_values('Reimbursed_Level', inplace=True)

In [74]:
reim

lof,Reimbursed_Level,-1,0,1
4,under 300,266380,131847,22165
2,300-1k,39787,20395,2761
0,1k-8k,32321,23007,2770
3,8k-20k,5360,6705,745
1,20k+,1567,2110,291


In [75]:
reim['Fraud'] = 100*reim['1']/(reim['0']+reim['1']+reim['-1'])
reim['Potential_Fraud'] = 100*reim['0']/(reim['0']+reim['1']+reim['-1'])
reim['non_Fraud'] = 100*reim['-1']/(reim['0']+reim['1']+reim['-1'])

In [76]:
fig1 = reim.plot.bar(x='Reimbursed_Level', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

In [78]:
fig1.show() # I save the plots static. If you do not want it, delete 'png'.

In [81]:
inpatient = df[df['Is_inpatient'] == 1]
outpatient = df[df['Is_inpatient'] == 0]

In [82]:
# #check NumOfClaimDays 
# in_claim_days = inpatient.groupby(['NumOfClaimDays', 'lof']).count().reset_index()
# in_claim_days = in_claim_days.pivot(index='NumOfClaimDays', columns='lof', values='Is_inpatient').reset_index()
# in_claim_days = in_claim_days.fillna(0)
# in_claim_days['Fraud'] = 100*in_claim_days['1']/(in_claim_days['0']+in_claim_days['1']+in_claim_days['-1'])
# in_claim_days['Potential_Fraud'] = 100*in_claim_days['0']/(in_claim_days['0']+in_claim_days['1']+in_claim_days['-1'])
# in_claim_days['non_Fraud'] = 100*in_claim_days['-1']/(in_claim_days['0']+in_claim_days['1']+in_claim_days['-1'])


# fig2 = in_claim_days.plot.bar(x='NumOfClaimDays', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig2.show()

In [ ]:
# #check numOfClaimDays for OutPatient
# out_claim_days = outpatient.groupby(['NumOfClaimDays', 'lof']).count().reset_index()
# out_claim_days = out_claim_days.pivot(index='NumOfClaimDays', columns='lof', values='Is_inpatient').reset_index()
# out_claim_days = out_claim_days.fillna(0)
# out_claim_days['Fraud'] = 100*out_claim_days['1']/(out_claim_days['0']+out_claim_days['1']+out_claim_days['-1'])
# out_claim_days['Potential_Fraud'] = 100*out_claim_days['0']/(out_claim_days['0']+out_claim_days['1']+out_claim_days['-1'])
# out_claim_days['non_Fraud'] = 100*out_claim_days['-1']/(out_claim_days['0']+out_claim_days['1']+out_claim_days['-1'])

# fig3 = out_claim_days.plot.bar(x='NumOfClaimDays', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig3.show('png')

In [86]:
#check NumOfPhys
in_physNum = inpatient.groupby(['PhysiciansNum', 'lof']).count().reset_index()
in_physNum = in_physNum.pivot(index='PhysiciansNum', columns='lof', values='Is_inpatient').reset_index()
in_physNum = in_physNum.fillna(0)
in_physNum['Fraud'] = 100*in_physNum['1']/(in_physNum['0']+in_physNum['1']+in_physNum['-1'])
in_physNum['Potential_Fraud'] = 100*in_physNum['0']/(in_physNum['0']+in_physNum['1']+in_physNum['-1'])
in_physNum['non_Fraud'] = 100*in_physNum['-1']/(in_physNum['0']+in_physNum['1']+in_physNum['-1'])

In [89]:
fig4 = in_physNum.plot.bar(x='PhysiciansNum', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

fig4.show()

In [92]:
in_physNum

lof,PhysiciansNum,-1,0,1,Fraud,Potential_Fraud,non_Fraud
0,0,53,44,9,8.490566,41.509434,50.000000
1,1,6341,7912,1012,6.629545,51.830986,41.539469
2,2,8933,11308,1451,6.689102,52.129817,41.181081
3,3,1745,1480,186,5.452946,43.389035,51.158018


In [93]:
out_physNum = outpatient.groupby(['PhysiciansNum', 'lof']).count().reset_index()
out_physNum = out_physNum.pivot(index='PhysiciansNum', columns='lof', values='Is_inpatient').reset_index()
out_physNum = out_physNum.fillna(0)
out_physNum['Fraud'] = 100*out_physNum['1']/(out_physNum['0']+out_physNum['1']+out_physNum['-1'])
out_physNum['Potential_Fraud'] = 100*out_physNum['0']/(in_physNum['0']+in_physNum['1']+out_physNum['-1'])
out_physNum['non_Fraud'] = 100*out_physNum['-1']/(out_physNum['0']+out_physNum['1']+out_physNum['-1'])

In [94]:
fig5 = out_physNum.plot.bar(x='PhysiciansNum', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

fig5.show()

In [129]:
# #check DiagnosisCode_Nu
# in_diag = inpatient.groupby(['DiagnosisCode_Num', 'lof']).count().reset_index()
# in_diag = in_diag.pivot(index='DiagnosisCode_Num', columns='lof', values='Is_inpatient').reset_index()
# in_diag = in_diag.fillna(0)
# in_diag['Fraud'] = 100*in_diag['1']/(in_diag['0']+in_diag['1']+in_diag['-1'])
# in_diag['Potential_Fraud'] = 100*in_diag['0']/(in_diag['0']+in_diag['1']+in_diag['-1'])
# in_diag['non_Fraud'] = 100*in_diag['-1']/(in_diag['0']+in_diag['1']+in_diag['-1'])

# fig6 = in_diag.plot.bar(x='DiagnosisCode_Num', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig6.show()

# out_diag = outpatient.groupby(['DiagnosisCode_Num', 'lof']).count().reset_index()
# out_diag = out_diag.pivot(index='DiagnosisCode_Num', columns='lof', values='Is_inpatient').reset_index()
# out_diag = out_diag.fillna(0)
# out_diag['Fraud'] = 100*out_diag['1']/(out_diag['0']+out_diag['1']+out_diag['-1'])
# out_diag['Potential_Fraud'] = 100*out_diag['0']/(out_diag['0']+out_diag['1']+out_diag['-1'])
# out_diag['non_Fraud'] = 100*out_diag['-1']/(out_diag['0']+out_diag['1']+out_diag['-1'])

# fig7 = out_diag.plot.bar(x='DiagnosisCode_Num', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig7.show()

In [99]:
#check ProcedureCode_Num
in_pro= inpatient.groupby(['ProcedureCode_Num', 'lof']).count().reset_index()
in_pro = in_pro.pivot(index='ProcedureCode_Num', columns='lof', values='Is_inpatient').reset_index()
in_pro = in_pro.fillna(0)
in_pro['Fraud'] = 100*in_pro['1']/(in_pro['0']+in_pro['1']+in_pro['-1'])
in_pro['Potential_Fraud'] = 100*in_pro['0']/(in_pro['0']+in_pro['1']+in_pro['-1'])
in_pro['non_Fraud'] = 100*in_pro['-1']/(in_pro['0']+in_pro['1']+in_pro['-1'])

In [100]:
fig8 = in_pro.plot.bar(x='ProcedureCode_Num', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

fig8.show()

In [101]:
out_pro= outpatient.groupby(['ProcedureCode_Num', 'lof']).count().reset_index()
out_pro = out_pro.pivot(index='ProcedureCode_Num', columns='lof', values='Is_inpatient').reset_index()
out_pro = out_pro.fillna(0)
out_pro['Fraud'] = 100*out_pro['1']/(out_pro['0']+out_pro['1']+out_pro['-1'])
out_pro['Potential_Fraud'] = 100*out_pro['0']/(out_pro['0']+out_pro['1']+out_pro['-1'])
out_pro['non_Fraud'] = 100*out_pro['-1']/(out_pro['0']+out_pro['1']+out_pro['-1'])

In [102]:
fig9 = out_pro.plot.bar(x='ProcedureCode_Num', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

fig9.show()

In [103]:
# check ProcedureCode on full data
pro= df.groupby(['ProcedureCode_Num', 'lof']).count().reset_index()
pro = pro.pivot(index='ProcedureCode_Num', columns='lof', values='Is_inpatient').reset_index()
pro = pro.fillna(0)
pro['Fraud'] = 100*pro['1']/(pro['0']+pro['1']+pro['-1'])
pro['Potential_Fraud'] = 100*pro['0']/(pro['0']+pro['1']+pro['-1'])
pro['non_Fraud'] = 100*pro['-1']/(pro['0']+pro['1']+pro['-1'])

In [104]:
fig10 = pro.plot.bar(x='ProcedureCode_Num', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

fig10.show()

In [105]:
# #check state 
# State  = df.groupby(['State', 'lof']).count().reset_index()
# State  = State.pivot(index='State', columns='lof', values='Race').reset_index()
# State  = State.fillna(0)
# State['Fraud'] = 100*State['1']/(State['0']+State['1']+State['-1'])
# State['Potential_Fraud'] = 100*State['0']/(State['0']+State['1']+State['-1'])
# State['non_Fraud'] = 100*State['-1']/(State['0']+State['1']+State['-1'])

# fig11 = State.plot.bar(x='State', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig11.show()

In [107]:
# #check admitDiagnose
# in_admit= inpatient.groupby(['AdmitDiagnosInDiagnos', 'lof']).count().reset_index()
# in_admit= in_admit.pivot(index='AdmitDiagnosInDiagnos', columns='lof', values='Is_inpatient').reset_index()
# in_admit= in_admit.fillna(0)
# in_admit['Fraud'] = 100*in_admit['1']/(in_admit['0']+in_admit['1']+in_admit['-1'])
# in_admit['Potential_Fraud'] = 100*in_admit['0']/(in_admit['0']+in_admit['1']+in_admit['-1'])
# in_admit['non_Fraud'] = 100*in_admit['-1']/(in_admit['0']+in_admit['1']+in_admit['-1'])

# fig12 = in_admit.plot.bar(x='AdmitDiagnosInDiagnos', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig12.show()

# #check admitDiagnose
# out_admit= outpatient.groupby(['AdmitDiagnosInDiagnos', 'lof']).count().reset_index()
# out_admit= out_admit.pivot(index='AdmitDiagnosInDiagnos', columns='lof', values='Is_inpatient').reset_index()
# out_admit= out_admit.fillna(0)
# out_admit['Fraud'] = 100*out_admit['1']/(out_admit['0']+out_admit['1']+out_admit['-1'])
# out_admit['Potential_Fraud'] = 100*out_admit['0']/(out_admit['0']+out_admit['1']+out_admit['-1'])
# out_admit['non_Fraud'] = 100*out_admit['-1']/(out_admit['0']+out_admit['1']+out_admit['-1'])

# fig13 = out_admit.plot.bar(x='AdmitDiagnosInDiagnos', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig13.show()

# #check admitDiagnose
# admit= df.groupby(['AdmitDiagnosInDiagnos', 'lof']).count().reset_index()
# admit= admit.pivot(index='AdmitDiagnosInDiagnos', columns='lof', values='Is_inpatient').reset_index()
# admit= admit.fillna(0)
# admit['Fraud'] = 100*admit['1']/(admit['0']+admit['1']+admit['-1'])
# admit['Potential_Fraud'] = 100*admit['0']/(admit['0']+admit['1']+admit['-1'])
# admit['non_Fraud'] = 100*admit['-1']/(admit['0']+admit['1']+admit['-1'])

# fig14 = admit.plot.bar(x='AdmitDiagnosInDiagnos', y=['Fraud',
#                                             'Potential_Fraud',
#                                             'non_Fraud'])

# fig14.show()

In [115]:
# #check age 
# age= df.groupby(['Age', 'lof']).count().reset_index()
# age= age.pivot(index='Age', columns='lof', values='Is_inpatient').reset_index()
# age= age.fillna(0)
# age['Fraud'] = 100*age['1']/(age['0']+age['1']+age['-1'])
# age['Potential_Fraud'] = 100*age['0']/(age['0']+age['1']+age['-1'])
# age['non_Fraud'] = 100*age['-1']/(age['0']+age['1']+age['-1'])

# fig15 = age.plot.bar(x='Age', y=['Fraud',
#                                 'Potential_Fraud',
#                                 'non_Fraud'])

# fig15.show()

In [128]:
# race= df.groupby(['Race', 'lof']).count().reset_index()
# race= race.pivot(index='Race', columns='lof', values='Is_inpatient').reset_index()
# race= race.fillna(0)
# race['Fraud'] = 100*race['1']/(race['0']+race['1']+race['-1'])
# race['Potential_Fraud'] = 100*race['0']/(race['0']+race['1']+race['-1'])
# race['non_Fraud'] = 100*race['-1']/(race['0']+race['1']+race['-1'])
# race

# fig16 = race.plot.bar(x='Race', y=['Fraud',
#                                 'Potential_Fraud',
#                                 'non_Fraud'])

# fig16.show()

In [127]:
# #check RenalDisease
# renal = df.groupby(['RenalDisease', 'lof']).count().reset_index()
# renal = renal.pivot(index='RenalDisease', columns='lof', values='Race').reset_index()
# renal = renal.fillna(0)
# renal['Fraud'] = 100*renal['1']/(renal['0']+renal['1']+renal['-1'])
# renal['Potential_Fraud'] = 100*renal['0']/(renal['0']+renal['1']+renal['-1'])
# renal['non_Fraud'] = 100*renal['-1']/(renal['0']+renal['1']+renal['-1'])
# renal

# fig17 = renal.plot.bar(x='RenalDisease', y=['Fraud',
#                                 'Potential_Fraud',
#                                 'non_Fraud'])

# fig17.show()